# `Preprocessing workflow`

#### The preprocessing workflow consists of ten steps:

![Preprocessing.png](images/Preprocessing.png)

At the end of the workflow, you can delete the interim directory.

Import libraries:

In [ ]:
import os
import shutil
import glob
from pathlib import Path
import numpy as np
import pandas as pd
from pyopenms import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'
import matplotlib

#### `1) PrecursorCorrection` (To the "highest intensity MS1 peak")

This algorithm is used directly after the file introduction, in order to correct any wrong MS1 precursor annotation. For each MS2 spectrum the corresponding MS1 spectrum is determined by using the RT information of the precursor. In the MS1 spectrum, the highest intensity peak is selected as the corrected precursor. We assume that, in a given mass window  (e.g. precursor mass +/- 10 ppm), the precursor with the hightest intensity was actually fragmented (top-n method), which is a method used in the Thermo Orbitrap instrument (Center for Biosustainability).


###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html//classOpenMS_1_1PrecursorCorrection.html#a8acf85ba8b9f249de0369bb083355982 

In [ ]:
results = "results"
if not os.path.exists(results):
    os.mkdir(results)

interim = os.path.join(results, "interim")
if not os.path.exists(interim):
    os.mkdir(interim)

path = os.path.join(interim, "mzML")

if os.path.exists(path):
    shutil.rmtree(path)
    os.mkdir(path)
else:
    os.mkdir(path)


input_original_files = glob.glob(os.path.join("data", "mzML", "*.mzML"))

for filename in input_original_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    delta_mzs = []
    mzs = []
    rts = []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    mzmlfile_path = os.path.join(path, "PCpeak_" + os.path.basename(filename))
    MzMLFile().store(mzmlfile_path, exp)

#### `2) Mass trace detection`

A mass trace extraction method that gathers peaks similar in m/z and moving along retention time.
Peaks of a MSExperiment are sorted by their intensity and stored in a list of potential chromatographic apex positions. Only peaks that are above the noise threshold (user-defined) are analyzed and only peaks that are n times above this minimal threshold are considered as apices. This saves computational resources and decreases the noise in the resulting output.
Starting with these, mass traces are extended in- and decreasingly in retention time. During this extension phase, the centroid m/z is computed on-line as an intensity-weighted mean of peaks.
The extension phase ends when either the frequency of gathered peaks drops below a threshold (min_sample_rate, see MassTraceDetection parameters) or when the number of missed scans exceeds a threshold (trace_termination_outliers, see MassTraceDetection parameters).
Finally, only mass traces that pass a filter (a certain minimal and maximal length as well as having the minimal sample rate criterion fulfilled) get added to the result.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1MassTraceDetection.html#abff6e392ce6da7af8f083397494a7971



#### `3) Elution peak detection`

Extracts chromatographic peaks from a mass trace.
Mass traces may consist of several consecutively (partly overlapping) eluting peaks, e.g., stemming from (almost) isobaric compounds that are separated by retention time. Especially in metabolomics, isomeric compounds with exactly the same mass but different retentional behaviour may still be contained in the same mass trace. This method first applies smoothing on the mass trace"s intensities, then detects local minima/maxima in order to separate the chromatographic peaks from each other. Depending on the "width_filtering" parameters, mass traces are filtered by length in seconds ("fixed" filter) or by quantile.

This method is in other words "deconvolution".

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1ElutionPeakDetection.htmldetails


#### `4) Feature detection`

FeatureFinderMetabo assembles metabolite features from singleton mass traces.
Mass traces alone would allow for further analysis such as metabolite ID or statistical evaluation. However, in general, monoisotopic mass traces are accompanied by satellite C13 peaks and thus may render the analysis more difficult. FeatureFinderMetabo fulfills a further data reduction step by assembling compatible mass traces to metabolite features (that is, all mass traces originating from one metabolite). To this end, multiple metabolite hypotheses are formulated and scored according to how well differences in RT (optional), m/z or intensity ratios match to those of theoretical isotope patterns.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1FeatureFindingMetabo.html

In [ ]:
path = os.path.join(interim, "Preprocessing")

if not os.path.exists(path): # if it doesn't exist
    os.mkdir(path) # create a path directory
else:
    shutil.rmtree(path)
    os.mkdir(path)

input_mzml_files = glob.glob(os.path.join(interim, "mzML", "PCpeak_*.mzML"))

# 2) Mass trace detection

feature_maps_FFM= []
for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e02)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 3) Elution peak detection (deconvolution)

    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 4) Feature finding metabo (isotope reduction)
  
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])  

    feature_maps_FFM.append(feature_map_FFM)
    featurefile = os.path.join(path, "FFM_" + os.path.basename(filename)[7:-5] +".featureXML")
    FeatureXMLFile().store(featurefile, feature_map_FFM)

#### `5) PrecursorCorrection (To the "nearest feature”)`

This algorithm is used after feature detection to allow for precursor correction on MS2 level. 

If there are MS2 spectra in the feature space which have been measured in isotope traces, it “corrects” the MS2 spectrum annotation to the monoisotopic trace. 

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1PrecursorCorrection.html

In [ ]:
path = os.path.join(interim, "mzML")

input_mzml_files = glob.glob(os.path.join(interim, "mzML", "PCpeak_*.mzML"))
input_feature_files = glob.glob(os.path.join(interim, "Preprocessing", "FFM_*.featureXML")) # or "FFM_*.featureXML" if you haven't filtered the blanks/control/QC

for mzml in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(mzml, exp)
    correct = PrecursorCorrection()

    for filename in input_feature_files:
        feature_map_MFD = FeatureMap()
        FeatureXMLFile().load(filename, feature_map_MFD)
        if os.path.basename(mzml)[7:-5] == os.path.basename(filename)[9:-11]:
            correct.correctToNearestFeature(feature_map_MFD, exp, 0.0, 100.0, True, False, False, False, 3, 0)
            corrected_file = os.path.join(path, "PCfeature_" + os.path.basename(mzml)[7:])
            MzMLFile().store(corrected_file, exp)

#### `6) MapAlignerPoseClustering `
This algorithm is used to perform a linear retention time alignment, in order to correct for chromatographic shifts in retention time. The reference file used for Map Alignment is the feature map with the highest number of features.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_MapAlignerPoseClustering.html

In [ ]:
path = os.path.join(interim, "Preprocessing")
interim= os.path.join("results", "interim")

input_feature_files = glob.glob(os.path.join(path, "FFM_*.featureXML"))
feature_maps=[]
for filename in input_feature_files:
    feature_map_MFD = FeatureMap()
    FeatureXMLFile().load(filename, feature_map_MFD)
    feature_maps.append(feature_map_MFD)

ref_index = [i[0] for i in sorted(enumerate([fm.size() for fm in feature_maps]), key=lambda x:x[1])][-1]

aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()

aligner_par.setValue("max_num_peaks_considered", -1)
aligner_par.setValue("superimposer:mz_pair_max_distance", 0.05)
aligner_par.setValue("pairfinder:distance_MZ:max_difference", 10.0)
aligner_par.setValue("pairfinder:distance_MZ:unit", "ppm")
aligner.setParameters(aligner_par)
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(feature_map, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True) # store original RT as meta value

for feature_map in feature_maps:    
    feature_file = os.path.join(path, "MapAligned_" + os.path.basename(feature_map.getMetaValue("spectra_data")[0].decode())[7:-5] +".featureXML")
    trafo_file= os.path.join(path, "MapAligned_" + os.path.basename(feature_map.getMetaValue("spectra_data")[0].decode())[7:-5] +".trafoXML")
    FeatureXMLFile().store(feature_file, feature_map)
    TransformationXMLFile().store(trafo_file, trafo)

Visualisation of data before and after alignment:

In [ ]:
import matplotlib.pyplot as plt

feature_maps = [feature_maps[ref_index]] + feature_maps[:ref_index] + feature_maps[ref_index+1:]

fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(1, 2, 1)
ax.set_title("consensus map before alignment")
ax.set_ylabel("m/z")
ax.set_xlabel("RT")

# use alpha value to display feature intensity
ax.scatter([f.getRT() for f in feature_maps[0]], [f.getMZ() for f in feature_maps[0]],
            alpha = np.asarray([f.getIntensity() for f in feature_maps[0]])/max([f.getIntensity() for f in feature_maps[0]]))

for fm in feature_maps[1:]:
    ax.scatter([f.getMetaValue("original_RT") for f in fm], [f.getMZ() for f in fm],
                alpha = np.asarray([f.getIntensity() for f in fm])/max([f.getIntensity() for f in fm]))

ax = fig.add_subplot(1,2,2)
ax.set_title("consensus map after alignment")
ax.set_xlabel("RT")

for fm in feature_maps:
    ax.scatter([f.getRT() for f in fm], [f.getMZ() for f in fm],
                alpha = np.asarray([f.getIntensity() for f in fm])/max([f.getIntensity() for f in fm]))

fig.tight_layout()
fig.show()

#### `7) MapAlignmentTransformer`
This algorithm is used to perform a linear retention time alignment, in order to correct for chromatographic shifts in retention time. Use the trafo XML files from the feature alignment and align the raw spectra.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1MapAlignmentTransformer.html

In [ ]:
input_mzml_files = sorted(glob.glob(os.path.join(interim, "mzML", "PCpeak_*.mzML")))
input_trafo = sorted(glob.glob(os.path.join(interim, "Preprocessing", "*.trafoXML")))

for filename in input_mzml_files:
    exp= MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    transformer = MapAlignmentTransformer()

    for trafo_XML in input_trafo:
        trafo=TransformationDescription()
        TransformationXMLFile().load(trafo_XML, trafo, True)

        if os.path.basename(trafo_XML)[11:-9] == os.path.basename(filename)[7:-5]:
            transformer.transformRetentionTimes(exp, trafo, True)
            mzml_file = os.path.join(interim, "mzML", "MapAligned_" + os.path.basename(exp.getLoadedFilePath())[7:-5] +".mzML")
            MzMLFile().store(mzml_file, exp)

#### `8) Metabolite adduct decharger` 

For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. Here, we do not save the file with neutral masses, but only the feature files that contain adduct annotations. You can add the list of adduct for the algorithm to parse through. SIRIUS, an algorithm that is later used, is only able to compute singly charged adducts so charges higher than 1 are filtered out. Use adduct list: [b"H-1:-:1", b"H-2O-1:0:0.05", b"CH2O2:0:0.5"] for negative mode, copy-pasting the following script:
```
input_feature_files= sorted(glob.glob(os.path.join(path, "MapAligned_*.featureXML")))

for filename in input_feature_files:
        feature_map = FeatureMap()    
        FeatureXMLFile().load(filename, feature_map)
        mfd = MetaboliteFeatureDeconvolution()
        mdf_par = mfd.getDefaults()
        mdf_par.setValue("negative_mode", "true")
        mdf_par.setValue("potential_adducts", [b"H-1:-:1", b"H-2O-1:0:0.5", b"CH2O2:0:0.5"])
        mdf_par.setValue("charge_min", -2, "Minimal possible charge")
        mdf_par.setValue("charge_max", 0, "Maximal possible charge")
        mdf_par.setValue("charge_span_max", 2)
        mdf_par.setValue("max_neutrals", 1)
        mdf_par.setValue("retention_max_diff", 3.0)
        mdf_par.setValue("retention_max_diff_local", 3.0)
        mfd.setParameters(mdf_par)
        feature_map_MFD = FeatureMap()
        cons_map0 = ConsensusMap()
        cons_map1 = ConsensusMap()
        mfd.compute(feature_map, feature_map_MFD, cons_map0, cons_map1)
        featurefile = os.path.join(path, "MFD_" + os.path.basename(filename)[11:])
        FeatureXMLFile().store(featurefile, feature_map_MFD)
```
###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/UTILS_MetaboliteAdductDecharger.html

In [ ]:
input_feature_files= sorted(glob.glob(os.path.join(path, "MapAligned_*.featureXML")))

for filename in input_feature_files:
        feature_map = FeatureMap()    
        FeatureXMLFile().load(filename, feature_map)
        mfd = MetaboliteFeatureDeconvolution()
        mdf_par = mfd.getDefaults()
        mdf_par.setValue("potential_adducts", [b"H:+:0.4",b"Na:+:0.2",b"NH4:+:0.2", b"H-1O-1:+:0.1", b"H-3O-2:+:0.1"])
        mdf_par.setValue("charge_min", 1, "Minimal possible charge")
        mdf_par.setValue("charge_max", 1, "Maximal possible charge")
        mdf_par.setValue("charge_span_max", 1)
        mdf_par.setValue("max_neutrals", 1)
        mdf_par.setValue("retention_max_diff", 3.0)
        mdf_par.setValue("retention_max_diff_local", 3.0)
        mfd.setParameters(mdf_par)
        feature_map_MFD = FeatureMap()
        cons_map0 = ConsensusMap()
        cons_map1 = ConsensusMap()
        mfd.compute(feature_map, feature_map_MFD, cons_map0, cons_map1)
        featurefile = os.path.join(path, "MFD_" + os.path.basename(filename)[11:])
        FeatureXMLFile().store(featurefile, feature_map_MFD)

#### `9) IDMapper` 

Introduce the features to a protein identification file (idXML)- the only way to annotate MS2 spectra for GNPS FBMN  (of later importance)

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IDMapper.html

In [ ]:
input_feature_files = sorted(glob.glob(os.path.join(path, "MFD_*.featureXML")))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

In [ ]:
use_centroid_rt= False
use_centroid_mz= True
protein_ids = []
peptide_ids= []

mapper = IDMapper()
input_mzml_files= sorted(glob.glob(os.path.join("results", "interim", "mzML", "MapAligned_*.mzML")))

for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    for fmap in feature_maps:
        peptide_ids = []
        protein_ids = []
        if os.path.basename(fmap.getMetaValue("spectra_data")[0].decode())[7:] == os.path.basename(filename)[11:]:
            mapper.annotate(fmap, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
        featureidx_file = os.path.join(path, "IDMapper_" + os.path.basename(fmap.getMetaValue("spectra_data")[0].decode())[7:-4] +"featureXML")
        FeatureXMLFile().store(featureidx_file, fmap)

#### `10) FeatureGroupingAlgorithmKD `

Feature linker clusters the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

In [ ]:
input_feature_files = glob.glob(os.path.join(interim, "Preprocessing", "IDMapper_*.featureXML"))
feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

In [ ]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue("spectra_data")[0].decode())[7:]
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setUniqueIds()
consensus_map.setColumnHeaders(file_descriptions)

Optionally filter consensus features with too many missing values.

In [ ]:
min_frac = 0.0 # minimum fraction of samples to be considered a valid consensus feature
# e.g. 0.5 -> values from at least 50% of samples required

# take consensus map from feature linking step as input...
print(f"Number of consensus features before filtering with threshold of {min_frac}: {consensus_map.size()}")

cm_filtered = ConsensusMap(consensus_map)
cm_filtered.clear(False)

n_samples = len(consensus_map.getColumnHeaders())

for cf in consensus_map:
    if len(cf.getFeatureList()) / n_samples >= min_frac:
        cm_filtered.push_back(cf)

print(f"Number of consensus features after filtering: {cm_filtered.size()}")

Optionally filter blank/QC/control consensus features with a mean blank intensity / mean sample intensity ration above a cutoff value (e.g. 30%).

In [ ]:
# Define the list of blank files
blank_files = []
# Define the intensity cutoff
cutoff = 0.3

print(f"Number of consensus features before blank filtering: {cm_filtered.size()}")

# Create a new ConsensusMap as a copy of cm
cm_blank_filtered = ConsensusMap(cm_filtered)

# Remove all the features but keep meta data
cm_blank_filtered.clear(False)

# Get a dict with map indeces and file names
headers = cm_filtered.getColumnHeaders()

# Get map indeces for blank files
blank_indices = []
for index, value in headers.items():
    if value.filename in blank_files:
        blank_indices.append(index)

# Iterate over the consensus features in the original consensus map
for cf in cm_filtered:
    # Get dict with index and intensity pairs from each feature in the consensus feature
    features = {f.getMapIndex(): f for f in cf.getFeatureList()}

    # Compute the average intensity in the blank files
    blank_intensities = [
        value.getIntensity() for key, value in features.items() if key in blank_indices
    ]
    if blank_intensities:
        blank_mean = np.mean(blank_intensities)
    else:
        blank_mean = 1

    # Compute the average intensity in the non-blank files
    non_blank_intensities = [
        value.getIntensity() for key, value in features.items() if key not in blank_indices
    ]
    if non_blank_intensities:
        non_blank_mean = np.mean(non_blank_intensities)
    else:
        non_blank_mean = 1  # set 1 to avoid division by zero error in the ratio

    # Compute the ratio between the two intensities (add one to avoid division by zero error)
    ratio = blank_mean / non_blank_mean

    # If the ratio is smaller than the cutoff, add the consensus feature to the new consensus map
    if ratio < cutoff:
        new_cf = ConsensusFeature(cf)
        new_cf.clear()
        for index, feature in features.items():
            if index in blank_indices:
                continue
            bf = BaseFeature()
            bf.setRT(feature.getRT())
            bf.setMZ(feature.getMZ())
            bf.setIntensity(feature.getIntensity())
            bf.setUniqueId(feature.getUniqueId())
            new_cf.insert(index, bf)
        cm_blank_filtered.push_back(new_cf)

# Set new column headers without blank files
cm_blank_filtered.setColumnHeaders({index: header for index, header in headers.items() if index not in blank_indices})

print(f"Number of consensus features after blank filtering: {cm_blank_filtered.size()}")

Save ConsensusMap to file

In [ ]:
Consensus_file= os.path.join(path, "consensus.consensusXML")
ConsensusXMLFile().store(Consensus_file, cm_blank_filtered)

Get consensus feature intensities as a dataframe

In [ ]:
consensus_map = ConsensusMap()
ConsensusXMLFile().load(Consensus_file, consensus_map)

df = consensus_map.get_df()
for cf in consensus_map:
    if cf.metaValueExists("best ion"):
        df["adduct"] = [cf.getMetaValue("best ion") for cf in consensus_map]
        break

df= df.reset_index()
df= df.drop(columns= ["sequence"])
df= df.rename(columns={"RT": "RT(s)", "mz" :"m/z"})

# annotate original feature IDs
fnames = [Path(value.filename).name for value in consensus_map.getColumnHeaders().values()]

ids = [[] for _ in fnames]

for cf in consensus_map:
    fids = {f.getMapIndex(): f.getUniqueId() for f in cf.getFeatureList()}
    for i, fname in enumerate(fnames):
        if i in fids.keys():
            ids[i].append(str(fids[i]))
        else:
            ids[i].append(pd.NA)

for i, f in enumerate(fnames):
    df[f"{fnames[i]}_IDs"] = ids[i]

feature_dir = os.path.join("results", "features")
if not os.path.exists(feature_dir):
    os.mkdir(feature_dir)

# store as tsv file
df.to_csv(os.path.join(feature_dir, "FeatureMatrix.tsv"), sep = "\t", index = False)
df

Plot:

In [ ]:
fig = px.scatter(df, x="RT(s)", y="m/z", color="quality")
fig.update_layout(title="Consensus features")
fig.show()